<h1 style="text-align: center; font-weight: bold;"><u>Import et chargement des données</u></h1>

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score
from scipy.stats import ks_2samp
from myFonctions import (
    get_cluster_labels,
    compute_ari_between_periods,
    compute_ks_test_between_periods,
    plot_ari,
    plot_feature_distribution
)
import warnings

In [2]:
# Chemin vers la base de données SQLite
db_path = 'data/olist.db'

# Créer une connexion à la base de données SQLite
conn = sqlite3.connect(db_path)

In [3]:
# 1. Extraire les dates min/max
query_min_max = """
SELECT MIN(order_purchase_timestamp) AS min_date,
       MAX(order_purchase_timestamp) AS max_date
FROM orders
"""
min_date, max_date = pd.read_sql_query(query_min_max, conn).iloc[0]
min_date = pd.to_datetime(min_date)
max_date = pd.to_datetime(max_date)

# 2. Générer tous les trimestres
trimestres = pd.period_range(start=min_date, end=max_date, freq='Q')

# 3. Identifier les trimestres avec au moins une commande
df_dates = pd.read_sql_query("SELECT order_purchase_timestamp FROM orders", conn)
df_dates['order_purchase_timestamp'] = pd.to_datetime(df_dates['order_purchase_timestamp'])
df_dates['period'] = df_dates['order_purchase_timestamp'].dt.to_period('Q')
trimestres_utiles = df_dates['period'].drop_duplicates().sort_values()

# 4. Générer le RFM pour chaque trimestre
rfm_all_periods = []

for period in trimestres_utiles:
    start = period.start_time
    end = period.end_time

    query = f"""
    WITH max_date AS (
        SELECT DATE('{end.date()}') AS global_max_date
    ),
    customer_orders AS (
        SELECT 
            c.customer_unique_id,
            o.order_id,
            o.order_purchase_timestamp
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_purchase_timestamp BETWEEN '{start.date()}' AND '{end.date()}'
    ),
    payments AS (
        SELECT 
            co.customer_unique_id,
            MAX(co.order_purchase_timestamp) AS last_order_date,
            COUNT(DISTINCT co.order_id) AS frequency,
            SUM(op.payment_value) AS monetary
        FROM customer_orders co
        JOIN order_pymts op ON co.order_id = op.order_id
        GROUP BY co.customer_unique_id
    ),
    rfm AS (
        SELECT 
            p.customer_unique_id,
            CAST(JULIANDAY(m.global_max_date) - JULIANDAY(p.last_order_date) AS INT) AS recency,
            p.frequency,
            ROUND(p.monetary, 2) AS monetary
        FROM payments p
        CROSS JOIN max_date m
    )

    SELECT *, '{period}' AS period
    FROM rfm
    """

    df_rfm = pd.read_sql_query(query, conn)
    rfm_all_periods.append(df_rfm)

rfm_all_periods = pd.concat(rfm_all_periods, ignore_index=True)

# 5. Calcul ARI et KS pour chaque paire de trimestres consécutifs
periods = sorted(rfm_all_periods['period'].unique())
results = []

for i in range(len(periods) - 1):
    period_1 = periods[i]
    period_2 = periods[i + 1]

    df1 = rfm_all_periods[rfm_all_periods['period'] == period_1].copy()
    df2 = rfm_all_periods[rfm_all_periods['period'] == period_2].copy()

    common_ids = set(df1.customer_unique_id).intersection(df2.customer_unique_id)
    df1 = df1[df1.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")
    df2 = df2[df2.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")

    if len(common_ids) < 5:
        continue  # ignorer si trop peu de clients communs

    scaler = StandardScaler()
    X1 = scaler.fit_transform(df1[['recency', 'frequency', 'monetary']])
    X2 = scaler.fit_transform(df2[['recency', 'frequency', 'monetary']])

    kmeans_1 = KMeans(n_clusters=4, random_state=42).fit(X1)
    kmeans_2 = KMeans(n_clusters=4, random_state=42).fit(X2)

    dbscan_1 = DBSCAN(eps=1.0, min_samples=5).fit(X1)
    dbscan_2 = DBSCAN(eps=1.0, min_samples=5).fit(X2)

    ari_kmeans = adjusted_rand_score(kmeans_1.labels_, kmeans_2.labels_)
    ari_dbscan = adjusted_rand_score(dbscan_1.labels_, dbscan_2.labels_)

    ks_results = {
        'recency': ks_2samp(df1['recency'], df2['recency']).pvalue,
        'frequency': ks_2samp(df1['frequency'], df2['frequency']).pvalue,
        'monetary': ks_2samp(df1['monetary'], df2['monetary']).pvalue,
    }

    results.append({
        'period_pair': f"{period_1} → {period_2}",
        'n_clients_common': len(common_ids),
        'ARI_kmeans': ari_kmeans,
        'ARI_dbscan': ari_dbscan,
        'KS_pvalue_recency': ks_results['recency'],
        'KS_pvalue_frequency': ks_results['frequency'],
        'KS_pvalue_monetary': ks_results['monetary'],
    })

df_eval = pd.DataFrame(results)
display(df_eval)


,period_pair,n_clients_common,ARI_kmeans,ARI_dbscan,KS_pvalue_recency,KS_pvalue_frequency,KS_pvalue_monetary
0,2017Q1 → 2017Q2,48,0.028896,0.226282,1.790296e-02,1.0,0.692660
1,2017Q2 → 2017Q3,97,0.038491,0.140347,7.071673e-02,1.0,0.963766
2,2017Q3 → 2017Q4,132,0.018704,0.246018,4.940724e-04,1.0,0.969610
3,2017Q4 → 2018Q1,137,0.028731,0.173639,1.088649e-03,1.0,0.860613
4,2018Q1 → 2018Q2,195,0.017444,0.125570,3.598142e-05,1.0,0.081057
5,2018Q2 → 2018Q3,102,0.004690,0.038812,6.693788e-09,1.0,0.994932


### 📊 Résumé des indicateurs d'évolution des clusters (RFM trimestriel)

| Indicateur             | Résultat observé                  | Interprétation                                                                 |
|------------------------|-----------------------------------|--------------------------------------------------------------------------------|
| **ARI - KMeans**       | Très faible (max ≈ 0.04)          | Les clients changent fréquemment de groupe, **clustering instable**.          |
| **ARI - DBSCAN**       | Faible à modéré (0.04 à 0.25)     | Un peu plus robuste que KMeans, mais **reste instable dans le temps**.        |
| **KS Test – Recency**  | p-values souvent < 0.05           | **Variation significative** dans la récence : les clients changent d'activité. |
| **KS Test – Frequency**| p-values ≈ 1.0                    | **Stable** : les clients achètent avec une fréquence similaire.               |
| **KS Test – Monetary** | p-values élevées (souvent > 0.6)  | **Stable** : les montants dépensés restent globalement constants.             |


📌 Analyse stratégique : passer à une analyse mensuelle ?
Les résultats montrent que :

Les comportements de fréquence et montant sont stables,

Mais les clusters restent instables à cause de changements fréquents dans la récence, ce qui affecte fortement le RFM global.

Cela soulève une question cruciale :
👉 Est-ce que l’agrégation trimestrielle est trop grossière pour capturer les dynamiques comportementales des clients ?

✅ Pourquoi tester le RFM tous les mois ?

Argument	Avantage attendu
Plus grande granularité temporelle	Permet de détecter plus tôt les changements d’habitude.
Meilleure détection des événements récents	Utile si le comportement client varie rapidement (ex : promotions, saisons).
Peut révéler des micro-segments plus cohérents	Certains segments peuvent être stables à court terme, mais invisibles à l’échelle trimestrielle.
Suivi plus précis de la récence	Comme c’est la dimension la plus instable, elle sera mieux mesurée mensuellement.

In [4]:
warnings.filterwarnings("ignore", message="ks_2samp: Exact calculation unsuccessful.*")
# 1. Extraire les dates min/max
query_min_max = """
SELECT MIN(order_purchase_timestamp) AS min_date,
       MAX(order_purchase_timestamp) AS max_date
FROM orders
"""
min_date, max_date = pd.read_sql_query(query_min_max, conn).iloc[0]
min_date = pd.to_datetime(min_date)
max_date = pd.to_datetime(max_date)

# 2. Générer tous les mois
mois = pd.period_range(start=min_date, end=max_date, freq='M')

# 3. Identifier les mois avec au moins une commande
df_dates = pd.read_sql_query("SELECT order_purchase_timestamp FROM orders", conn)
df_dates['order_purchase_timestamp'] = pd.to_datetime(df_dates['order_purchase_timestamp'])
df_dates['period'] = df_dates['order_purchase_timestamp'].dt.to_period('M')
mois_utiles = df_dates['period'].drop_duplicates().sort_values()

# 4. Générer le RFM pour chaque mois
rfm_all_periods = []

for period in mois_utiles:
    start = period.start_time
    end = period.end_time

    query = f"""
    WITH max_date AS (
        SELECT DATE('{end.date()}') AS global_max_date
    ),
    customer_orders AS (
        SELECT 
            c.customer_unique_id,
            o.order_id,
            o.order_purchase_timestamp
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_purchase_timestamp BETWEEN '{start.date()}' AND '{end.date()}'
    ),
    payments AS (
        SELECT 
            co.customer_unique_id,
            MAX(co.order_purchase_timestamp) AS last_order_date,
            COUNT(DISTINCT co.order_id) AS frequency,
            SUM(op.payment_value) AS monetary
        FROM customer_orders co
        JOIN order_pymts op ON co.order_id = op.order_id
        GROUP BY co.customer_unique_id
    ),
    rfm AS (
        SELECT 
            p.customer_unique_id,
            CAST(JULIANDAY(m.global_max_date) - JULIANDAY(p.last_order_date) AS INT) AS recency,
            p.frequency,
            ROUND(p.monetary, 2) AS monetary
        FROM payments p
        CROSS JOIN max_date m
    )

    SELECT *, '{period}' AS period
    FROM rfm
    """

    df_rfm = pd.read_sql_query(query, conn)
    rfm_all_periods.append(df_rfm)

rfm_all_periods = pd.concat(rfm_all_periods, ignore_index=True)

# 5. Calcul ARI et KS pour chaque paire de mois consécutifs
periods = sorted(rfm_all_periods['period'].unique())
results = []

for i in range(len(periods) - 1):
    period_1 = periods[i]
    period_2 = periods[i + 1]

    df1 = rfm_all_periods[rfm_all_periods['period'] == period_1].copy()
    df2 = rfm_all_periods[rfm_all_periods['period'] == period_2].copy()

    common_ids = set(df1.customer_unique_id).intersection(df2.customer_unique_id)
    df1 = df1[df1.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")
    df2 = df2[df2.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")

    if len(common_ids) < 5:
        continue  # ignorer si trop peu de clients communs

    scaler = StandardScaler()
    X1 = scaler.fit_transform(df1[['recency', 'frequency', 'monetary']])
    X2 = scaler.fit_transform(df2[['recency', 'frequency', 'monetary']])

    kmeans_1 = KMeans(n_clusters=4, random_state=42).fit(X1)
    kmeans_2 = KMeans(n_clusters=4, random_state=42).fit(X2)

    dbscan_1 = DBSCAN(eps=1.0, min_samples=5).fit(X1)
    dbscan_2 = DBSCAN(eps=1.0, min_samples=5).fit(X2)

    ari_kmeans = adjusted_rand_score(kmeans_1.labels_, kmeans_2.labels_)
    ari_dbscan = adjusted_rand_score(dbscan_1.labels_, dbscan_2.labels_)

    ks_results = {
        'recency': ks_2samp(df1['recency'], df2['recency']).pvalue,
        'frequency': ks_2samp(df1['frequency'], df2['frequency']).pvalue,
        'monetary': ks_2samp(df1['monetary'], df2['monetary']).pvalue,
    }

    results.append({
        'period_pair': f"{period_1} → {period_2}",
        'n_clients_common': len(common_ids),
        'ARI_kmeans': ari_kmeans,
        'ARI_dbscan': ari_dbscan,
        'KS_pvalue_recency': ks_results['recency'],
        'KS_pvalue_frequency': ks_results['frequency'],
        'KS_pvalue_monetary': ks_results['monetary'],
    })

df_eval = pd.DataFrame(results)
display(df_eval)

,period_pair,n_clients_common,ARI_kmeans,ARI_dbscan,KS_pvalue_recency,KS_pvalue_frequency,KS_pvalue_monetary
0,2017-03 → 2017-04,13,0.014440,0.082353,0.126488,1.000000,0.999212
1,2017-04 → 2017-05,14,-0.086567,-0.136553,0.635485,1.000000,0.999592
2,2017-05 → 2017-06,18,0.070729,0.061350,0.132394,1.000000,0.971540
3,2017-06 → 2017-07,15,0.021253,0.184994,0.678138,1.000000,0.938331
4,2017-07 → 2017-08,23,0.076579,0.072168,0.024720,0.999999,0.421782
5,2017-08 → 2017-09,31,0.048171,-0.073979,0.079120,1.000000,0.823454
6,2017-09 → 2017-10,30,0.089063,0.096886,0.392945,1.000000,0.807963
7,2017-10 → 2017-11,36,-0.010073,-0.064352,0.706867,1.000000,0.510011
8,2017-11 → 2017-12,40,0.132505,0.153288,0.000066,1.000000,0.918805
9,2017-12 → 2018-01,19,-0.084507,-0.118145,0.537929,0.978078,0.537929


| Période               | Clients Communs | ARI KMeans | ARI DBSCAN | RFM Change ?      | KMeans Stable ? | DBSCAN Stable ? | Conclusion Synthétique                                       |
|-----------------------|------------------|------------|------------|-------------------|------------------|------------------|----------------------------------------------------------------|
| 2017-03 → 2017-04     | 13               | 0.014      | 0.082      | 🟢 Stable         | ❌ Faible        | ❌ Faible        | Clusters très instables, pas de changement RFM                |
| 2017-04 → 2017-05     | 14               | -0.087     | -0.137     | 🟢 Stable         | ❌ Très faible   | ❌ Très faible   | Instabilité des deux modèles sans évolution comportementale    |
| 2017-05 → 2017-06     | 18               | 0.071      | 0.061      | 🟢 Stable         | ⚠️ Moyenne       | ⚠️ Moyenne       | Légère stabilité, comportement clients constant                |
| 2017-06 → 2017-07     | 15               | 0.021      | 0.185      | 🟢 Stable         | ❌ Faible        | ⚠️ Moyenne       | DBSCAN un peu plus cohérent que KMeans                        |
| 2017-07 → 2017-08     | 23               | 0.077      | 0.072      | 🔥 Recency        | ⚠️ Moyenne       | ⚠️ Moyenne       | Début de changement RFM, légers ajustements de cluster        |
| 2017-08 → 2017-09     | 31               | 0.048      | -0.074     | 🟢 Stable         | ⚠️ Faible        | ❌ Très faible   | Faible cohérence, surtout DBSCAN                             |
| 2017-09 → 2017-10     | 30               | 0.089      | 0.097      | 🟢 Stable         | ⚠️ Moyenne       | ⚠️ Moyenne       | Stabilité modérée dans les deux approches                     |
| 2017-10 → 2017-11     | 36               | -0.010     | -0.064     | 🟢 Stable         | ❌ Très faible   | ❌ Très faible   | Instabilité injustifiée dans les deux modèles                 |
| 2017-11 → 2017-12     | 40               | 0.133      | 0.153      | 🔥 Recency        | ⚠️ Moyenne       | ⚠️ Moyenne       | Récence change, stabilité moyenne                            |
| 2017-12 → 2018-01     | 19               | -0.085     | -0.118     | 🟢 Stable         | ❌ Faible        | ❌ Faible        | Cluster incohérent sans changement de fond                    |
| 2018-01 → 2018-02     | 27               | -0.005     | -0.101     | 🟢 Stable         | ❌ Faible        | ❌ Faible        | Faible stabilité malgré comportement stable                   |
| 2018-02 → 2018-03     | 23               | -0.071     | -0.009     | 🟢 Stable         | ❌ Faible        | ⚠️ Moyenne       | Léger mieux pour DBSCAN, mais pas convaincant                 |
| 2018-03 → 2018-04     | 34               | 0.316      | 0.234      | 🟢 Stable         | ✅ Bonne         | ✅ Bonne         | Excellente cohérence dans les deux modèles                    |
| 2018-04 → 2018-05     | 44               | 0.083      | -0.053     | 🔥 Recency        | ⚠️ Moyenne       | ❌ Faible        | DBSCAN perd en cohérence malgré rupture récence              |
| 2018-05 → 2018-06     | 38               | 0.008      | -0.061     | 🟢 Stable         | ❌ Faible        | ❌ Faible        | Incohérence persistante, pas de comportement nouveau          |
| 2018-06 → 2018-07     | 21               | 0.180      | 0.102      | 🟢 Stable         | ⚠️ Moyenne       | ⚠️ Moyenne       | Meilleure tenue des clusters, stabilité comportementale       |
| 2018-07 → 2018-08     | 48               | 0.034      | 0.498      | 🔥 Recency        | ❌ Faible        | ✅ Bonne         | DBSCAN réagit mieux à la rupture comportementale              |
| 2018-08 → 2018-09     | 8                | 0.270      | 1.000      | 🟢 Stable         | ✅ Bonne         | ✅ Excellente     | Très forte stabilité, mais effectif trop faible pour conclure |


In [5]:
# 1. Extraire les dates min/max
query_min_max = """
SELECT MIN(order_purchase_timestamp) AS min_date,
       MAX(order_purchase_timestamp) AS max_date
FROM orders
"""
min_date, max_date = pd.read_sql_query(query_min_max, conn).iloc[0]
min_date = pd.to_datetime(min_date)
max_date = pd.to_datetime(max_date)

# 2. Générer des bornes de périodes tous les 15 jours
quinzaines = pd.date_range(start=min_date, end=max_date, freq='15D')

# 3. Vérifier les périodes avec commandes
df_dates = pd.read_sql_query("SELECT order_purchase_timestamp FROM orders", conn)
df_dates['order_purchase_timestamp'] = pd.to_datetime(df_dates['order_purchase_timestamp'])

# 4. Générer le RFM pour chaque période de 15 jours
rfm_all_periods = []

for i in range(len(quinzaines) - 1):
    start = quinzaines[i]
    end = quinzaines[i + 1]

    # Vérifier présence de commandes
    has_orders = df_dates[
        (df_dates['order_purchase_timestamp'] >= start) &
        (df_dates['order_purchase_timestamp'] < end)
    ]

    if has_orders.empty:
        continue

    query = f"""
    WITH max_date AS (
        SELECT DATE('{end.date()}') AS global_max_date
    ),
    customer_orders AS (
        SELECT 
            c.customer_unique_id,
            o.order_id,
            o.order_purchase_timestamp
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_purchase_timestamp BETWEEN '{start.date()}' AND '{end.date()}'
    ),
    payments AS (
        SELECT 
            co.customer_unique_id,
            MAX(co.order_purchase_timestamp) AS last_order_date,
            COUNT(DISTINCT co.order_id) AS frequency,
            SUM(op.payment_value) AS monetary
        FROM customer_orders co
        JOIN order_pymts op ON co.order_id = op.order_id
        GROUP BY co.customer_unique_id
    ),
    rfm AS (
        SELECT 
            p.customer_unique_id,
            CAST(JULIANDAY(m.global_max_date) - JULIANDAY(p.last_order_date) AS INT) AS recency,
            p.frequency,
            ROUND(p.monetary, 2) AS monetary
        FROM payments p
        CROSS JOIN max_date m
    )

    SELECT *, '{start.date()} → {end.date()}' AS period
    FROM rfm
    """

    df_rfm = pd.read_sql_query(query, conn)
    rfm_all_periods.append(df_rfm)

# Fusionner tous les RFM
rfm_all_periods = pd.concat(rfm_all_periods, ignore_index=True)

# 5. Évaluer la stabilité entre périodes consécutives
periods = sorted(rfm_all_periods['period'].unique())
results = []

for i in range(len(periods) - 1):
    period_1 = periods[i]
    period_2 = periods[i + 1]

    df1 = rfm_all_periods[rfm_all_periods['period'] == period_1].copy()
    df2 = rfm_all_periods[rfm_all_periods['period'] == period_2].copy()

    common_ids = set(df1.customer_unique_id).intersection(df2.customer_unique_id)
    df1 = df1[df1.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")
    df2 = df2[df2.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")

    if len(common_ids) < 5:
        continue  # ignorer si trop peu de clients communs

    scaler = StandardScaler()
    X1 = scaler.fit_transform(df1[['recency', 'frequency', 'monetary']])
    X2 = scaler.fit_transform(df2[['recency', 'frequency', 'monetary']])

    # Clustering
    kmeans_1 = KMeans(n_clusters=4, random_state=42).fit(X1)
    kmeans_2 = KMeans(n_clusters=4, random_state=42).fit(X2)

    dbscan_1 = DBSCAN(eps=1.0, min_samples=5).fit(X1)
    dbscan_2 = DBSCAN(eps=1.0, min_samples=5).fit(X2)

    # Stabilité des clusters
    ari_kmeans = adjusted_rand_score(kmeans_1.labels_, kmeans_2.labels_)
    ari_dbscan = adjusted_rand_score(dbscan_1.labels_, dbscan_2.labels_)

    # Test KS pour la stabilité des distributions
    ks_results = {
        'recency': ks_2samp(df1['recency'], df2['recency']).pvalue,
        'frequency': ks_2samp(df1['frequency'], df2['frequency']).pvalue,
        'monetary': ks_2samp(df1['monetary'], df2['monetary']).pvalue,
    }

    results.append({
        'period_pair': f"{period_1} → {period_2}",
        'n_clients_common': len(common_ids),
        'ARI_kmeans': ari_kmeans,
        'ARI_dbscan': ari_dbscan,
        'KS_pvalue_recency': ks_results['recency'],
        'KS_pvalue_frequency': ks_results['frequency'],
        'KS_pvalue_monetary': ks_results['monetary'],
    })

# 6. Résultat final
df_eval = pd.DataFrame(results)
display(df_eval)

,period_pair,n_clients_common,ARI_kmeans,ARI_dbscan,KS_pvalue_recency,KS_pvalue_frequency,KS_pvalue_monetary
0,2017-02-01 → 2017-02-16 → 2017-02-16 → 2017-03-03,8,0.026087,1.000000,0.660140,1.000000,0.660140
1,2017-03-18 → 2017-04-02 → 2017-04-02 → 2017-04-17,5,-0.111111,1.000000,1.000000,1.000000,1.000000
2,2017-04-02 → 2017-04-17 → 2017-04-17 → 2017-05-02,5,-0.111111,1.000000,0.357143,1.000000,1.000000
3,2017-04-17 → 2017-05-02 → 2017-05-02 → 2017-05-17,7,-0.037037,1.000000,0.575175,0.999961,0.962704
4,2017-05-02 → 2017-05-17 → 2017-05-17 → 2017-06-01,9,-0.020243,-0.012500,0.125874,1.000000,0.730111
5,2017-05-17 → 2017-06-01 → 2017-06-01 → 2017-06-16,6,0.285714,1.000000,0.474026,1.000000,0.930736
6,2017-06-01 → 2017-06-16 → 2017-06-16 → 2017-07-01,5,-0.111111,1.000000,1.000000,1.000000,0.079365
7,2017-07-01 → 2017-07-16 → 2017-07-16 → 2017-07-31,12,0.030651,0.022222,0.099547,1.000000,0.536098
8,2017-07-16 → 2017-07-31 → 2017-07-31 → 2017-08-15,11,0.036558,0.000000,0.479150,1.000000,0.074661
9,2017-07-31 → 2017-08-15 → 2017-08-15 → 2017-08-30,9,0.172414,0.000000,0.351707,1.000000,0.989469


In [7]:
# 1. Extraire les dates min/max
query_min_max = """
SELECT MIN(order_purchase_timestamp) AS min_date,
       MAX(order_purchase_timestamp) AS max_date
FROM orders
"""
min_date, max_date = pd.read_sql_query(query_min_max, conn).iloc[0]
min_date = pd.to_datetime(min_date)
max_date = pd.to_datetime(max_date)

# 2. Générer des bornes de périodes tous les 15 jours
quinzaines = pd.date_range(start=min_date, end=max_date, freq='5D')

# 3. Vérifier les périodes avec commandes
df_dates = pd.read_sql_query("SELECT order_purchase_timestamp FROM orders", conn)
df_dates['order_purchase_timestamp'] = pd.to_datetime(df_dates['order_purchase_timestamp'])

# 4. Générer le RFM pour chaque période de 15 jours
rfm_all_periods = []

for i in range(len(quinzaines) - 1):
    start = quinzaines[i]
    end = quinzaines[i + 1]

    # Vérifier présence de commandes
    has_orders = df_dates[
        (df_dates['order_purchase_timestamp'] >= start) &
        (df_dates['order_purchase_timestamp'] < end)
    ]

    if has_orders.empty:
        continue

    query = f"""
    WITH max_date AS (
        SELECT DATE('{end.date()}') AS global_max_date
    ),
    customer_orders AS (
        SELECT 
            c.customer_unique_id,
            o.order_id,
            o.order_purchase_timestamp
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_purchase_timestamp BETWEEN '{start.date()}' AND '{end.date()}'
    ),
    payments AS (
        SELECT 
            co.customer_unique_id,
            MAX(co.order_purchase_timestamp) AS last_order_date,
            COUNT(DISTINCT co.order_id) AS frequency,
            SUM(op.payment_value) AS monetary
        FROM customer_orders co
        JOIN order_pymts op ON co.order_id = op.order_id
        GROUP BY co.customer_unique_id
    ),
    rfm AS (
        SELECT 
            p.customer_unique_id,
            CAST(JULIANDAY(m.global_max_date) - JULIANDAY(p.last_order_date) AS INT) AS recency,
            p.frequency,
            ROUND(p.monetary, 2) AS monetary
        FROM payments p
        CROSS JOIN max_date m
    )

    SELECT *, '{start.date()} → {end.date()}' AS period
    FROM rfm
    """

    df_rfm = pd.read_sql_query(query, conn)
    rfm_all_periods.append(df_rfm)

# Fusionner tous les RFM
rfm_all_periods = pd.concat(rfm_all_periods, ignore_index=True)

# 5. Évaluer la stabilité entre périodes consécutives
periods = sorted(rfm_all_periods['period'].unique())
results = []

for i in range(len(periods) - 1):
    period_1 = periods[i]
    period_2 = periods[i + 1]

    df1 = rfm_all_periods[rfm_all_periods['period'] == period_1].copy()
    df2 = rfm_all_periods[rfm_all_periods['period'] == period_2].copy()

    common_ids = set(df1.customer_unique_id).intersection(df2.customer_unique_id)
    df1 = df1[df1.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")
    df2 = df2[df2.customer_unique_id.isin(common_ids)].set_index("customer_unique_id")

    if len(common_ids) < 5:
        continue  # ignorer si trop peu de clients communs

    scaler = StandardScaler()
    X1 = scaler.fit_transform(df1[['recency', 'frequency', 'monetary']])
    X2 = scaler.fit_transform(df2[['recency', 'frequency', 'monetary']])

    # Clustering
    kmeans_1 = KMeans(n_clusters=4, random_state=42).fit(X1)
    kmeans_2 = KMeans(n_clusters=4, random_state=42).fit(X2)

    dbscan_1 = DBSCAN(eps=1.0, min_samples=5).fit(X1)
    dbscan_2 = DBSCAN(eps=1.0, min_samples=5).fit(X2)

    # Stabilité des clusters
    ari_kmeans = adjusted_rand_score(kmeans_1.labels_, kmeans_2.labels_)
    ari_dbscan = adjusted_rand_score(dbscan_1.labels_, dbscan_2.labels_)

    # Test KS pour la stabilité des distributions
    ks_results = {
        'recency': ks_2samp(df1['recency'], df2['recency']).pvalue,
        'frequency': ks_2samp(df1['frequency'], df2['frequency']).pvalue,
        'monetary': ks_2samp(df1['monetary'], df2['monetary']).pvalue,
    }

    results.append({
        'period_pair': f"{period_1} → {period_2}",
        'n_clients_common': len(common_ids),
        'ARI_kmeans': ari_kmeans,
        'ARI_dbscan': ari_dbscan,
        'KS_pvalue_recency': ks_results['recency'],
        'KS_pvalue_frequency': ks_results['frequency'],
        'KS_pvalue_monetary': ks_results['monetary'],
    })

# 6. Résultat final
df_eval = pd.DataFrame(results)
display(df_eval)

C:\Users\david\AppData\Local\Temp\ipykernel_10660\1387553555.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rfm_all_periods = pd.concat(rfm_all_periods, ignore_index=True)


,period_pair,n_clients_common,ARI_kmeans,ARI_dbscan,KS_pvalue_recency,KS_pvalue_frequency,KS_pvalue_monetary
0,2017-11-23 → 2017-11-28 → 2017-11-28 → 2017-12-03,6,-0.250000,1.0,1.000000,1.000000,0.930736
1,2018-01-02 → 2018-01-07 → 2018-01-07 → 2018-01-12,5,-0.111111,1.0,0.873016,1.000000,0.873016
2,2018-05-02 → 2018-05-07 → 2018-05-07 → 2018-05-12,7,-0.037037,1.0,0.962704,0.999961,0.575175
3,2018-07-26 → 2018-07-31 → 2018-07-31 → 2018-08-05,5,-0.111111,1.0,0.873016,1.000000,0.873016


| 🧠 Conclusion Globale                                         | ✅ Interprétation                                                                                                                                  |
|--------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| ✔️ Distributions RFM stables                                 | Les tests de Kolmogorov-Smirnov montrent que les variables `recency`, `frequency` et `monetary` ne varient pas significativement entre périodes. |
| ❌ Clusters (KMeans) instables                               | Les indices ARI sont faibles ou négatifs → faible cohérence des regroupements entre périodes.                                                     |
| ➤ Hypothèse 1 : Échantillon trop petit                      | Peu de clients communs (5 à 7) → peu de données pour former des clusters fiables.                                                                 |
| ➤ Hypothèse 2 : Comportement client trop changeant          | Les clients peuvent avoir des comportements très variables à l’échelle de 5 jours.                                                                |
| ➤ Hypothèse 3 : Clustering inadapté                         | KMeans suppose des clusters sphériques et équilibrés, ce qui n’est peut-être pas le cas ici.                                                      |
| ✔️ Clusters (DBSCAN) plus stables                            | DBSCAN montre une stabilité des regroupements (ARI = 1), ce qui indique une meilleure capacité à identifier des structures sous-jacentes.         |
| ➤ Hypothèse 4 : DBSCAN détecte des structures non sphériques | Contrairement à KMeans, DBSCAN peut mieux gérer des structures complexes (clusters de formes variées) sans faire d'hypothèses sur leur forme.    |
